# Preparación de Datos con PySpark

Este notebook realiza la preparación de datos usando PySpark para procesar documentos PDF.

In [4]:
# Configuración inicial
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import udf, col
import numpy as np
import subprocess

# Instalar pdf2image si no está disponible
try:
    import pdf2image
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pdf2image"])
finally:
    from pdf2image import convert_from_path

import cv2

# Configurar Spark
print("Configurando Spark...")
spark = SparkSession.builder \
    .appName("PDFPreprocessing") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()
print("Spark configurado correctamente.")

# Configurar directorios
RAW_DIR = 'data/raw'
PROCESSED_DIR = 'data/processed'

# Validar y crear directorios si no existen
if not os.path.exists(RAW_DIR):
    print(f"Directorio {RAW_DIR} no existe. Creándolo...")
    os.makedirs(RAW_DIR, exist_ok=True)
else:
    print(f"Directorio {RAW_DIR} ya existe.")

if not os.path.exists(PROCESSED_DIR):
    print(f"Directorio {PROCESSED_DIR} no existe. Creándolo...")
    os.makedirs(PROCESSED_DIR, exist_ok=True)
else:
    print(f"Directorio {PROCESSED_DIR} ya existe.")

print(f"Directorios configurados: RAW_DIR={RAW_DIR}, PROCESSED_DIR={PROCESSED_DIR}")

Configurando Spark...
Spark configurado correctamente.
Directorio data/raw ya existe.
Directorio data/processed ya existe.
Directorios configurados: RAW_DIR=data/raw, PROCESSED_DIR=data/processed


In [5]:
# Procesar PDFs a imágenes
def process_pdf(pdf_path):
    """Procesa un PDF y guarda sus páginas como imágenes"""
    try:
        print(f"Procesando {pdf_path}...")
        # Convertir PDF a imágenes
        pages = convert_from_path(pdf_path)
        pdf_name = os.path.basename(pdf_path).replace('.pdf', '')
        
        # Crear directorio para este PDF
        pdf_dir = os.path.join(PROCESSED_DIR, pdf_name)
        os.makedirs(pdf_dir, exist_ok=True)
        
        # Guardar cada página como imagen
        for i, page in enumerate(pages):
            page_path = os.path.join(pdf_dir, f'page_{i}.png')
            page.save(page_path)
            print(f"Guardada página {i} de {pdf_name} en {page_path}")
            
        print(f"Procesamiento de {pdf_name} completado.")
    except Exception as e:
        print(f"Error procesando {pdf_path}: {str(e)}")

# Procesar todos los PDFs en el directorio RAW_DIR
pdf_files = [os.path.join(RAW_DIR, f) for f in os.listdir(RAW_DIR) if f.endswith('.pdf')]
for pdf_file in pdf_files:
    process_pdf(pdf_file)

Procesando data/raw\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30.pdf...
Guardada página 0 de 2-TITULOS-15-DE-NOVIEMBRE-2024-1-30 en data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_0.png
Guardada página 1 de 2-TITULOS-15-DE-NOVIEMBRE-2024-1-30 en data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_1.png
Guardada página 2 de 2-TITULOS-15-DE-NOVIEMBRE-2024-1-30 en data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_2.png
Guardada página 3 de 2-TITULOS-15-DE-NOVIEMBRE-2024-1-30 en data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_3.png
Guardada página 4 de 2-TITULOS-15-DE-NOVIEMBRE-2024-1-30 en data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_4.png
Guardada página 5 de 2-TITULOS-15-DE-NOVIEMBRE-2024-1-30 en data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_5.png
Guardada página 6 de 2-TITULOS-15-DE-NOVIEMBRE-2024-1-30 en data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_6.png
Guardada página 7 de 2-TITULOS-15-DE-NOVIEMBRE-2024-1-30 en data/processed\2-TITULO

In [1]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import json
import logging
import os

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class PDFLabeler:
    def __init__(self, processed_dir='data/processed', labels_file='data/labels.json'):
        self.processed_dir = processed_dir
        self.labels_file = labels_file
        self.window = tk.Tk()
        self.window.title("PDF Page Labeler")
        self.setup_gui()
        self.load_existing_labels()

    def setup_gui(self):
        """Configura la interfaz gráfica"""
        # Frame principal
        self.main_frame = ttk.Frame(self.window, padding="10")
        self.main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

        # Área de visualización
        self.image_label = ttk.Label(self.main_frame)
        self.image_label.grid(row=0, column=0, columnspan=3, pady=5)

        # Controles
        ttk.Label(self.main_frame, text="PDF actual:").grid(
            row=1, column=0, sticky=tk.W)
        self.pdf_label = ttk.Label(self.main_frame, text="")
        self.pdf_label.grid(row=1, column=1, columnspan=2, sticky=tk.W)

        ttk.Label(self.main_frame, text="Página:").grid(
            row=2, column=0, sticky=tk.W)
        self.page_label = ttk.Label(self.main_frame, text="")
        self.page_label.grid(row=2, column=1, columnspan=2, sticky=tk.W)

        # Botones
        ttk.Button(self.main_frame, text="Es Primera Página",
                   command=self.mark_first_page).grid(row=3, column=0, pady=5)
        ttk.Button(self.main_frame, text="No es Primera Página",
                   command=self.next_page).grid(row=3, column=1, pady=5)
        ttk.Button(self.main_frame, text="Guardar y Salir",
                   command=self.save_and_exit).grid(row=3, column=2, pady=5)
        # Agregar botones de navegación
        nav_frame = ttk.Frame(self.main_frame)
        nav_frame.grid(row=4, column=0, columnspan=3, pady=5)

        ttk.Button(nav_frame, text="← Anterior",
                   command=self.previous_page).grid(row=0, column=0, padx=5)
        ttk.Button(nav_frame, text="Siguiente →",
                   command=self.next_page).grid(row=0, column=1, padx=5)

        # Mostrar estado de etiquetado
        self.status_label = ttk.Label(self.main_frame, text="")
        self.status_label.grid(row=5, column=0, columnspan=3, pady=5)

        # Agregar atajos de teclado
        self.window.bind('<Left>', lambda e: self.previous_page())
        self.window.bind('<Right>', lambda e: self.next_page())
        self.window.bind('<space>', lambda e: self.mark_first_page())
        self.window.bind('q', lambda e: self.save_and_exit())

    def previous_page(self):
        """Retrocede a la página anterior"""
        if self.current_page > 0:
            self.current_page -= 1
            self.show_current_page()

    def show_current_page(self):
        """Muestra la página actual"""
        try:
            if self.current_pdf_index >= len(self.pdfs):
                return  # No intentar mostrar una página si ya hemos terminado

            page_path = os.path.join(
                self.processed_dir,
                self.current_pdf,
                f'page_{self.current_page}.png'
            )

            if not os.path.exists(page_path):
                raise FileNotFoundError(
                    f"No se encuentra la página: {page_path}")

            # Cargar y mostrar imagen
            image = Image.open(page_path)

            # Mantener proporción de aspecto
            width, height = image.size
            ratio = min(800/width, 600/height)
            new_size = (int(width * ratio), int(height * ratio))
            image = image.resize(new_size, Image.Resampling.LANCZOS)

            photo = ImageTk.PhotoImage(image)
            self.image_label.configure(image=photo)
            self.image_label.image = photo

            # Actualizar etiquetas
            self.pdf_label.configure(text=self.current_pdf)
            self.page_label.configure(
                text=f"Página {self.current_page + 1} de {self.get_total_pages()}")

            # Actualizar estado
            is_marked = self.is_page_marked(self.current_page)
            status = "MARCADA COMO PRIMERA PÁGINA" if is_marked else "NO MARCADA"
            self.status_label.configure(text=f"Estado: {status}")

        except Exception as e:
            logger.error(f"Error mostrando página: {e}")
            self.status_label.configure(text="Error mostrando la página")

    def get_total_pages(self):
        """Obtiene el total de páginas del PDF actual"""
        pdf_dir = os.path.join(self.processed_dir, self.current_pdf)
        return len([f for f in os.listdir(pdf_dir) if f.endswith('.png')])

    def is_page_marked(self, page_num):
        """Verifica si la página está marcada como primera página"""
        return (self.current_pdf in self.labels and
                page_num in self.labels[self.current_pdf]["target_pages"])

    def load_existing_labels(self):
        """Carga etiquetas existentes si las hay"""
        try:
            if os.path.exists(self.labels_file):
                with open(self.labels_file, 'r') as f:
                    self.labels = json.load(f)
            else:
                self.labels = {}
        except Exception as e:
            logger.error(f"Error cargando etiquetas: {e}")
            self.labels = {}

    def start_labeling(self):
        """Inicia el proceso de etiquetado"""
        pdfs = [d for d in os.listdir(self.processed_dir)
                if os.path.isdir(os.path.join(self.processed_dir, d))]

        if not pdfs:
            logger.error("No se encontraron PDFs procesados")
            return

        self.pdfs = pdfs
        self.current_pdf_index = 0
        self.current_pdf = self.pdfs[self.current_pdf_index]
        self.current_page = 0
        self.show_current_page()
        self.window.mainloop()

    def mark_first_page(self):
        """Marca la página actual como primera página"""
        if self.current_pdf not in self.labels:
            self.labels[self.current_pdf] = {
                "target_pages": [], "total_pages": 0}

        if self.current_page not in self.labels[self.current_pdf]["target_pages"]:
            self.labels[self.current_pdf]["target_pages"].append(
                self.current_page)
            logger.info(
                f"Página {self.current_page} marcada como primera página")

        self.next_page()

    def next_page(self):
        """Pasa a la siguiente página"""
        self.current_page += 1
        pages = os.listdir(os.path.join(self.processed_dir, self.current_pdf))

        if self.current_page >= len(pages):
            self.labels[self.current_pdf]["total_pages"] = len(pages)
            self.save_labels()
            self.load_next_pdf()
        else:
            self.show_current_page()

    def load_next_pdf(self):
        """Carga el siguiente PDF no etiquetado"""
        self.current_pdf_index += 1

        if self.current_pdf_index < len(self.pdfs):
            self.current_pdf = self.pdfs[self.current_pdf_index]
            self.current_page = 0
            self.show_current_page()
        else:
            logger.info("Todos los PDFs han sido etiquetados")
            self.save_and_exit()

    def save_labels(self):
        """Guarda las etiquetas en el archivo"""
        try:
            with open(self.labels_file, 'w') as f:
                json.dump(self.labels, f, indent=4)
            logger.info("Etiquetas guardadas correctamente")
        except Exception as e:
            logger.error(f"Error guardando etiquetas: {e}")

    def save_and_exit(self):
        """Guarda las etiquetas y cierra la aplicación"""
        self.save_labels()
        self.window.after(1000, self.window.quit)  # Esperar un segundo antes de cerrar

if __name__ == "__main__":
    labeler = PDFLabeler()
    labeler.start_labeling()

INFO:__main__:Etiquetas guardadas correctamente
ERROR:__main__:Error mostrando página: No se encuentra la página: data/processed\processed_images.parquet\page_0.png


In [2]:
# Paso 4: Crear un DataFrame de Spark con las Imágenes y Etiquetas
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col
import cv2
import numpy as np

# Configurar Spark
print("Configurando Spark...")
spark = SparkSession.builder \
    .appName("ImageProcessing") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()
print("Spark configurado correctamente.")

PROCESSED_DIR = 'data/processed'
LABELS_JSON = 'data/labels.json'

def preprocess_image(image_path, target_size=(224, 224)):
    """Preprocesa una imagen para el modelo"""
    try:
        print(f"Preprocesando imagen {image_path}...")
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        resized = cv2.resize(image, target_size)
        normalized = resized.astype(np.float32) / 255.0
        return normalized.flatten().tolist()
    except Exception as e:
        print(f"Error preprocesando imagen {image_path}: {str(e)}")
        return None

# Leer etiquetas desde el archivo JSON
with open(LABELS_JSON, 'r') as f:
    labels = json.load(f)

# Crear DataFrame de imágenes y etiquetas
image_data = []
for pdf_name, pdf_info in labels.items():
    for page_num in range(pdf_info["total_pages"]):
        image_path = os.path.join(PROCESSED_DIR, pdf_name, f'page_{page_num}.png')
        label = 1 if page_num in pdf_info["target_pages"] else 0
        features = preprocess_image(image_path)
        if features is not None:
            image_data.append((image_path, label, features))

schema = StructType([
    StructField("path", StringType(), False),
    StructField("label", IntegerType(), False),
    StructField("features", ArrayType(FloatType()), True)
])

df = spark.createDataFrame(image_data, schema)
print("DataFrame creado correctamente.")

# Guardar DataFrame procesado
output_path = os.path.join(PROCESSED_DIR, "processed_images_with_labels.parquet")
print(f"Guardando DataFrame en {output_path}...")
df.write.mode("overwrite").parquet(output_path)
print("DataFrame guardado correctamente.")

# Mostrar estadísticas del procesamiento
print(f"Total de imágenes procesadas: {df.count()}")
print("\nDistribución de etiquetas:")
df.groupBy("label").count().show()

# Verificar valores nulos o problemas
print("\nVerificación de calidad:")
df.summary().show()

Configurando Spark...
Spark configurado correctamente.
Preprocesando imagen data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_0.png...
Preprocesando imagen data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_1.png...
Preprocesando imagen data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_2.png...
Preprocesando imagen data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_3.png...
Preprocesando imagen data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_4.png...
Preprocesando imagen data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_5.png...
Preprocesando imagen data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_6.png...
Preprocesando imagen data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_7.png...
Preprocesando imagen data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_8.png...
Preprocesando imagen data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_9.png...
Preprocesando imagen data/processed\2-TITULOS-15-DE-NOVIEMBRE-2024-1-30\page_10.png..

In [9]:
import os
import sys
import json
import cv2
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, udf
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StandardScaler

# Configuración del entorno
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Configuración de rutas
PROCESSED_DIR = 'data/processed'
LABELS_JSON = 'data/labels.json'
MODEL_DIR = 'data/models'
os.makedirs(MODEL_DIR, exist_ok=True)

# Inicializar Spark
spark = SparkSession.builder \
    .appName("DocumentClassifier") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.sql.shuffle.partitions", "10") \
    .getOrCreate()

def preprocess_image(image_path, target_size=(224, 224)):
    try:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise ValueError(f"No se pudo cargar la imagen: {image_path}")
        resized = cv2.resize(image, target_size)
        normalized = resized.astype(np.float32) / 255.0
        return normalized.flatten().tolist()
    except Exception as e:
        print(f"Error procesando {image_path}: {str(e)}")
        return None

# Cargar y procesar datos
with open(LABELS_JSON, 'r') as f:
    labels = json.load(f)

image_data = []
for pdf_name, pdf_info in labels.items():
    for page_num in range(pdf_info["total_pages"]):
        image_path = os.path.join(PROCESSED_DIR, pdf_name, f'page_{page_num}.png')
        if os.path.exists(image_path):
            label = 1 if page_num in pdf_info["target_pages"] else 0
            features = preprocess_image(image_path)
            if features is not None:
                image_data.append((image_path, label, features))

# Crear DataFrame
schema = StructType([
    StructField("path", StringType(), False),
    StructField("label", IntegerType(), False),
    StructField("features", ArrayType(FloatType()), True)
])

df = spark.createDataFrame(image_data, schema)

# Convertir features a vector
array_to_vector_udf = udf(lambda x: Vectors.dense(x), VectorUDT())
df = df.withColumn("features_vector", array_to_vector_udf("features"))

# Escalar características
scaler = StandardScaler(inputCol="features_vector", 
                       outputCol="scaled_features",
                       withStd=True,
                       withMean=True)
scaler_model = scaler.fit(df)
df_scaled = scaler_model.transform(df)

# Preparar datos para entrenamiento
final_df = df_scaled.select("label", "scaled_features")
train_df, test_df = final_df.randomSplit([0.8, 0.2], seed=42)

# Entrenar modelo
lr = LogisticRegression(
    featuresCol="scaled_features",
    labelCol="label",
    maxIter=20,
    regParam=0.1,
    elasticNetParam=0.8
)

lr_model = lr.fit(train_df)

# Evaluar modelo
predictions = lr_model.transform(test_df)
evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)

# Métricas
accuracy = evaluator.evaluate(predictions)
print(f"\nMétricas del modelo:")
print(f"Accuracy: {accuracy:.4f}")

# Guardar modelo y datos procesados
lr_model.save(os.path.join(MODEL_DIR, "logistic_regression_model"))
df_scaled.write.mode("overwrite").parquet(os.path.join(PROCESSED_DIR, "processed_scaled_data.parquet"))

# Mostrar estadísticas
print("\nEstadísticas del procesamiento:")
print(f"Total imágenes: {df.count()}")
print("\nDistribución de clases:")
df.groupBy("label").count().show()

spark.stop()


Métricas del modelo:
Accuracy: 0.6000

Estadísticas del procesamiento:
Total imágenes: 30

Distribución de clases:
+-----+-----+
|label|count|
+-----+-----+
|    0|   27|
|    1|    3|
+-----+-----+



In [6]:
!pip freeze > requirements.txt

In [5]:
# echo $SPARK_HOME  # En Linux/Mac
!echo %SPARK_HOME% 
# En Windows

C:\Program Files\spark-3.5.4-bin-hadoop3


In [8]:
!python -c "import numpy"

In [6]:
!pip install numpy
import numpy as np
print(np.__version__)

2.0.2
